<a href="https://colab.research.google.com/github/paulvoliva/Public-PDE-Solvers/blob/main/Black_Scholes_NCDE_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 708.0MB 26kB/s 
     |████████████████████████████████| 5.9MB 52.9MB/s 
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
  Found existing installation: torchvision 0.8.1+cu101
    Uninstalling torchvision-0.8.1+cu101:
      Successfully uninstalled torchvision-0.8.1+cu101


In [2]:
!pip install ray==1.0.1.post1

     |████████████████████████████████| 23.1MB 117kB/s 
     |████████████████████████████████| 204kB 59.8MB/s 
     |████████████████████████████████| 71kB 11.3MB/s 
     |████████████████████████████████| 133kB 63.9MB/s 
     |████████████████████████████████| 1.3MB 48.4MB/s 
     |████████████████████████████████| 3.2MB 39.6MB/s 
     |████████████████████████████████| 71kB 11.4MB/s 
     |████████████████████████████████| 81kB 12.4MB/s 
     |████████████████████████████████| 61kB 10.1MB/s 
     |████████████████████████████████| 296kB 55.7MB/s 
     |████████████████████████████████| 143kB 60.0MB/s 
     |████████████████████████████████| 102kB 13.9MB/s 
  Created wheel for gpustat: filename=gpustat-0.6.0-cp36-none-any.whl size=12622 sha256=93b02af420f670dae1abd15a3327505f85dec927e4717c53292e5aaf2d59f773
  Stored in directory: /root/.cache/pip/wheels/48/b4/d5/fb5b7f1d040f2ff20687e3bad6867d63155dbde5a7c10f4293
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl 

In [3]:
!pip install tensorboardX

     |████████████████████████████████| 317kB 17.8MB/s 


In [4]:
pip install torchdiffeq

In [5]:
pip install signatory

     |████████████████████████████████| 61kB 8.0MB/s 
  Created wheel for signatory: filename=signatory-1.2.4.1.7.1-cp36-cp36m-linux_x86_64.whl size=6424607 sha256=faa101e698cbbb872b935b4f3498c5f76730b9f73ee73a29a71cf33006558edb
  Stored in directory: /root/.cache/pip/wheels/84/6e/dc/5762aa0bebc28a8a6bba71e3eff188534a95dadd45402a3328
Successfully built signatory


In [ ]:
from dataset import Path, FixedCDEDataset, FlexibleCDEDataset, SubsampleDataset
from scalers import TrickScaler
from intervals import FixedIntervalSampler, RandomSampler, BatchIntervalSampler, create_interval_dataloader
from functions import torch_ffill
from rdeint import rdeint
from model import NeuralRDE
import torch
import math
from ray import tune
from ray.tune.schedulers import ASHAScheduler
import os
import signatory
from itertools import product

'''Given a data tensor of shape [N, L, C] that is filled with nan values, and a corresponding times tensor of shape
    [N, L] the corresponds to the time the data was collected for each row
    
    The data of shape [N, L, C]. It is assumed that the times are in the first index of the
    data channels.
    
    N = number of paths
    L = time dim
    C = channels'''

'''
# General Form of our problem:
\begin{equation}
\left\{\begin{array}{ll}
u_{t}-\sum_{i=1}^d \partial_i (\sum_{j=1}^d a_{ij} \partial_j u) + \sum_{i=1}^d b_i \partial_i u + cu = f, & \text { in } \Omega \times[0, T] \\
u(x, t)=g(x, t), & \text { on } \partial \Omega \times[0, T] \\
u(x, 0)=h(x), & \text { in } \Omega
\end{array}\right.
\end{equation}
where $x$ is a d-dimensional vector
'''

'''
Here we are solving the Black-Scholes equation with $r=0$:
\begin{equation}
\left\{\begin{array}{ll}
\partial_t V + \frac{1}{2} \sigma^2 S^2 \partial^2_S V = 0 & \text{on} (0,100)\times(0,100)\\
V(0,t) = 0\\
V(100,t) = 100\\
V(S,100) = \max{S-20,0}
\end{array}\right.
\end{equation}
'''

'''
# I use this to get the shape of the tensor when I debug
old_repr = torch.Tensor.__repr__
def tensor_info(tensor):
    return repr(tensor.shape)[6:] + ' ' + repr(tensor.dtype)[6:] + '@' + str(tensor.device) + '\n' + old_repr(tensor)
torch.Tensor.__repr__ = tensor_info
#'''

def train(config, checkpoint_dir=None):

    # setting to cuda
    if torch.cuda.is_available():
      dev = "cuda:0"
    else:
      dev = "cpu"

    device = torch.device(dev)
    #print(device)

    ''' # Parameters'''

    npaths = 20
    intervals = 30
    step = 10
    t_mesh_size = int(step*intervals)
    batch_size = npaths
    depth = 1

    assert npaths==batch_size, "Warning! The code does not yet support actual batches of data, modify the training function to do so"

    bnpaths = 20 #number of paths on the boundary (will be multiplied by 4)

    up = 100
    down = 0
    T = 100             #remember to divide this by 100 in all of the functions
    T0 = 0
    dim = 1

    ''' # Dataset '''

    x = torch.Tensor(npaths, t_mesh_size, 1).uniform_(down, up).to(device).requires_grad_(True)
    t = torch.linspace(T0, T, t_mesh_size).unsqueeze(1).unsqueeze(2).view(1, t_mesh_size, 1).repeat(npaths, 1, 1).to(device).requires_grad_(True)

    X = [x, t]

    xv = x[:, ::step, :].clone().detach().to(device).requires_grad_(True)
    tv = t[:, ::step, :].clone().detach().to(device).requires_grad_(True)

    XV = [xv, tv]

    tx = torch.cat((t, x), dim=2).to(device)

    dataset = FlexibleCDEDataset(tx, torch.ones_like(tx)[:, :, 0], depth=depth)
    sampler = FixedIntervalSampler(t_mesh_size, step, from_start=True, include_end=False)

    dataloader = create_interval_dataloader(dataset, sampler, batch_size)

    # boundary points

    ones = torch.ones(bnpaths, t_mesh_size, 1)
    t1, t2 = torch.Tensor(bnpaths, t_mesh_size, 1).uniform_(T0, T), torch.Tensor(bnpaths, t_mesh_size, 1).uniform_(T0, T)

    bl, br = torch.cat((t1, down*ones), dim=2), torch.cat((t2, up*ones), dim=2)
    btx = torch.cat((bl, br), dim=0)

    borderset = FlexibleCDEDataset(btx, torch.ones_like(btx)[:, :, 0], depth=depth)
    borderloader = create_interval_dataloader(borderset, sampler, 2*bnpaths)

    ''' # PDE functions '''

    def func_f(x, t):
        return(torch.zeros(1,1))

    def func_g(bx, bt):
        # bx, by, bt denote the boundary coordinates
        return bx

    def func_h(x):
        h,i = torch.max(torch.cat(((x-20).unsqueeze(2), torch.zeros(x.size()).unsqueeze(2).to(device)), dim=2), dim=2)
        return h

    def func_c(y_output_u):
        return 0

    # this is meant to be a d by d-dimensional array containing npaths by intervals by 1 tensors
    a = torch.ones(1, 1, npaths, intervals, 1).to(device)

    # this is meant to be a d-dimensional containing npaths by intervals by 1 tensors
    b = torch.zeros(1, npaths, intervals, 1).to(device)

    x_setup = xv.clone().detach().to(device)
    t_setup = tv.clone().detach().to(device)
    btx_setup = btx[:, ::step, :].clone().detach()

    h = func_h(x_setup[:, -1, :]).to(device)
    f = func_f(x_setup, t_setup).to(device)
    g = func_g(btx_setup[:, :, 1], btx_setup[:, :, 0]).unsqueeze(2).to(device)

    def func_w(x):  # returns 1 for positions in the domain and 0 otherwise
        w_bool = torch.gt(1 - torch.abs(x), torch.zeros(x.shape).to(device)) & torch.gt(torch.abs(x), torch.zeros(x.shape).to(device))
        w_val = torch.where(w_bool, 1 - torch.abs(x) + torch.abs(x), torch.zeros(x.shape).to(device))
        return w_val

    ''' # Model'''

    # The generator model is just the NeuralRDE model

    class discriminator(torch.nn.Module):  # this makes the v function
        def __init__(self, config):
            super().__init__()
            self.num_layers = config['v_layers']
            self.hidden_dim = config['v_hidden_dim']
            self.input = torch.nn.Linear(dim+1, self.hidden_dim)
            self.hidden = torch.nn.Linear(self.hidden_dim, self.hidden_dim)
            self.output = torch.nn.Linear(self.hidden_dim, 1)
            self.net = torch.nn.Sequential(*[
                self.input,
                *[torch.nn.ReLU(), self.hidden] * self.num_layers,
                torch.nn.Tanh(),
                self.output

            ])

        def forward(self, inp):
            x = self.net(inp)
            return x

        def backward(self, retain_graph=True):
            self.loss.backward(retain_graph=retain_graph)
            return (self.loss)

    ''' # Loss '''

    def I(y_output_u, y_output_v, XV, X, a=a, b=b,h=h, f=f, c=func_c):
        y_output_u.retain_grad()
        y_output_v.retain_grad()
        phi = y_output_v * func_w(xv)
        y_output_u.backward(torch.ones_like(y_output_u), retain_graph=True)
        du = {}
        for i in range(dim):
            du['du_'+str(i)] = X[i].grad[:, ::step, :]
        phi.backward(torch.ones_like(phi), retain_graph=True)
        dphi = {}
        for i in range(dim+1):
            dphi['dphi_'+str(i)] = XV[i].grad
        s1 = y_output_u[:, -1, :] * phi[:, -1, :] - h
        s2 = (y_output_u * dphi['dphi_1'])/t_mesh_size*step  # for t does this make sense?
        s31 = 0
        for i,j in product(range(dim), repeat=2):
            s31 += a[i, j, :, :, :] * dphi['dphi_'+str(i)] * du['du_'+str(j)]
        s32 = 0
        for i in range(dim):
            s32 += b[i] * phi * du['du_'+str(i)]
        s3 = (T-T0)*(s31 + s32 + func_c(y_output_u) * y_output_u * phi - f * phi)/t_mesh_size*step
        I = torch.sum(s1 - torch.sum(s2 - s3, 1), 0)
        for i in X:
            i.grad.data.zero_()
        for i in XV:
            i.grad.data.zero_()
        return I


    def L_init(y_output_u, h=h):
        return torch.mean((y_output_u[:, -1, :] - h )** 2)


    def L_bdry(u_net, border_logsig, g=g):
        return torch.mean((u_net(border_logsig) - g) ** 2)


    def L_int(y_output_u, y_output_v, XV=XV, X=X):
        # x needs to be the set of points set plugged into net_u and net_v
        return torch.log((I(y_output_u, y_output_v, XV, X)) ** 2) - torch.log(torch.sum(y_output_v ** 2))


    def Loss_u(y_output_u, y_output_v, border_logsig, u_net, alpha, gamma):
        return L_int(y_output_u, y_output_v) + gamma * L_init(y_output_u) + alpha * L_bdry(
            u_net, border_logsig)

    def Loss_v(y_output_u, y_output_v):
        return -L_int(y_output_u, y_output_v)

    iteration = 2000

    (initial, logsig), responses = next(iter(dataloader))
    logsig_dim = logsig.shape[2]

    '''
    batch, reponses = next(iter(dataloader))
    batchborder, responses = next(iter(borderloader))
    '''

    n1 = config['n1']
    n2 = config['n2']

    # neural network models
    u_net = NeuralRDE(dim+1, logsig_dim, config['u_hidden_dim'], 1, hidden_hidden_dim=config['u_hidden_hidden_dim'], num_layers=config['u_layers'], return_sequences=True).to(device)
    v_net = discriminator(config).to(device)

    # optimizers for WAN
    optimizer_u = torch.optim.Adam(u_net.parameters(), lr=config['u_rate'])
    optimizer_v = torch.optim.Adam(v_net.parameters(), lr=config['v_rate'])

    v_inp = XV[-1]
    for i in range(dim):
        v_inp = torch.cat((v_inp, XV[i]), dim=2)

    for batch, responses in dataloader:
        prediction_u = u_net(batch)
    prediction_v = v_net(v_inp)

    Loss = 0

    for k in range(iteration):

        for i in range(n1):
            for batch, responses in dataloader:
                for batchborder, responses in borderloader:
                    (initial, logsig), (binitial, blogsig) = batch, batchborder
                    batch, batchborder = (initial.to(device), logsig.to(device)), (binitial.to(device), blogsig.to(device))
                    loss_u = Loss_u(prediction_u, prediction_v, batchborder, u_net, config['alpha'], config['alpha'])
                    optimizer_u.zero_grad()
                    loss_u.backward(retain_graph=True)
                    optimizer_u.step()
                    prediction_u = u_net(batch)

        for j in range(n2):
            loss_v = Loss_v(prediction_u, prediction_v)
            optimizer_v.zero_grad()
            loss_v.backward(retain_graph=True)
            optimizer_v.step()
            prediction_v = v_net(v_inp)

        Loss += 0.1*loss_u

        if k % 10 == 0:
            print(k, loss_u.data, loss_v.data)
            #error_test = torch.mean(
                #torch.sqrt(torch.square((func_u_sol(x[:, ::step, :], y[:, ::step, :], t[:, ::step, :]) - prediction_u.data)))).data
            #print("error test " + str(error_test))

            tune.report(Loss=Loss.item())

            with tune.checkpoint_dir(k) as checkpoint_dir:
                path = os.path.join(checkpoint_dir, "checkpoint")
                torch.save((u_net.state_dict(), v_net.state_dict()), path)
            '''
config = {
    'v_layers': 5,
    'v_hidden_dim': 10,
    'u_hidden_dim': 15,
    'u_hidden_hidden_dim': 5,
    'u_layers': 5,
    'alpha': 25,  # 1e5*boundary_sample_size*4 # 25
    'n1': 2,
    'n2': 1,
    'u_rate': 0.0015,       # 0.0015
    'v_rate': 0.0015       # 0.0015
}
'''
config = {
    'v_layers': tune.qrandint(2, 10),   # 5
    'v_hidden_dim': tune.qrandint(5, 20),  # 10
    'u_hidden_dim': tune.qrandint(5, 20),
    'u_hidden_hidden_dim': tune.qrandint(5, 20),
    'u_layers': tune.qrandint(5, 20),
    'alpha': tune.loguniform(10, 1e7),  # 1e5*boundary_sample_size*4 # 25
    'n1': 2,
    'n2': 1,
    'u_rate': tune.loguniform(1e-6, 1e-2),       # 0.0015
    'v_rate': tune.loguniform(1e-6, 1e-2)       # 0.0015
}
'''

train(config)

'''
tune.utils.diagnose_serialization(train)

analysis = tune.run(
    train,
    num_samples=200,
    scheduler=ASHAScheduler(metric="Loss", mode="min", grace_period=10, max_t=200, reduction_factor=4),
    config=config,
    verbose=2,
    resources_per_trial={'cpu':1}
)

best_trial = analysis.get_best_trial(metric="Loss", mode="min")
print("Best trial config: {}".format(best_trial.config))
'''

best_trained_generator = NeuralRDE(3, logsig_dim, best_trial.config['u_hidden_dim'], 1, hidden_hidden_dim=best_trial.config['u_hidden_hidden_dim'], num_layers=best_trial.config['u_layers'], return_sequences=True)
best_trained_discriminator = discriminator(best_trial.config)

best_checkpoint_dir = best_trial.checkpoint.value
generator_state, discriminator_state = torch.load(os.path.join(
    best_checkpoint_dir, "checkpoint"))
'''


Trying to serialize <function train at 0x7f6338be4f28>...
Serialization succeeded!


2021-01-27 11:15:31,462	INFO services.py:1092 -- View the Ray dashboard at http://127.0.0.1:8265
2021-01-27 11:15:34,049	WARNING tune.py:401 -- Tune detects GPUs, but no trials are using GPUs. To enable trials to use GPUs, set tune.run(resources_per_trial={'gpu': 1}...) which allows Tune to expose 1 GPU to each trial. You can also override `Trainable.default_resource_request` if using the Trainable API.


Trial name,status,loc,alpha,u_hidden_dim,u_hidden_hidden_dim,u_layers,u_rate,v_hidden_dim,v_layers,v_rate
train_f9627_00000,RUNNING,,738645,13,9,11,4.07248e-05,17,3,3.01163e-05


(pid=314) /usr/local/lib/python3.6/dist-packages/torchdiffeq/_impl/misc.py:11: UserWarning: Midpoint: Unexpected arguments {'eps': 1e-05}
(pid=314)   warnings.warn('{}: Unexpected arguments {}'.format(solver.__class__.__name__, unused_kwargs))
(pid=315) /usr/local/lib/python3.6/dist-packages/torchdiffeq/_impl/misc.py:11: UserWarning: Midpoint: Unexpected arguments {'eps': 1e-05}
(pid=315)   warnings.warn('{}: Unexpected arguments {}'.format(solver.__class__.__name__, unused_kwargs))


Result for train_f9627_00000:
  Loss: 498951904.0
  date: 2021-01-27_11-15-36
  done: false
  experiment_id: a2ee3c2d0cce484bbec9a9348e21f57e
  experiment_tag: 0_alpha=7.3865e+05,u_hidden_dim=13,u_hidden_hidden_dim=9,u_layers=11,u_rate=4.0725e-05,v_hidden_dim=17,v_layers=3,v_rate=3.0116e-05
  hostname: fc555e4772b4
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  pid: 314
  time_since_restore: 0.956587553024292
  time_this_iter_s: 0.956587553024292
  time_total_s: 0.956587553024292
  timestamp: 1611746136
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: f9627_00000
  
(pid=314) 0 tensor([4.9895e+09]) tensor([-8.3574])
Result for train_f9627_00001:
  Loss: 276555.03125
  date: 2021-01-27_11-15-36
  done: false
  experiment_id: 7fc901853453456588329fbe1b363afd
  experiment_tag: 1_alpha=552.21,u_hidden_dim=8,u_hidden_hidden_dim=12,u_layers=11,u_rate=1.3543e-06,v_hidden_dim=5,v_layers=6,v_rate=4.7049e-05
  hostname: fc555e4772b4
  iterations_since_restore: 1
  node_ip:

...

(pid=314) 30 tensor([4.7376e+09]) tensor([-8.5175])
Result for train_f9627_00001:
  Loss: 8565285.0
  date: 2021-01-27_11-16-00
  done: false
  experiment_id: 7fc901853453456588329fbe1b363afd
  experiment_tag: 1_alpha=552.21,u_hidden_dim=8,u_hidden_hidden_dim=12,u_layers=11,u_rate=1.3543e-06,v_hidden_dim=5,v_layers=6,v_rate=4.7049e-05
  hostname: fc555e4772b4
  iterations_since_restore: 4
  node_ip: 172.28.0.2
  pid: 315
  should_checkpoint: true
  time_since_restore: 24.556297302246094
  time_this_iter_s: 7.82333517074585
  time_total_s: 24.556297302246094
  timestamp: 1611746160
  timesteps_since_restore: 0
  training_iteration: 4
  trial_id: f9627_00001
  
(pid=315) 30 tensor([2760442.5000]) tensor([-8.4532])
Result for train_f9627_00000:
  Loss: 19766620160.0
  date: 2021-01-27_11-16-07
  done: false
  experiment_id: a2ee3c2d0cce484bbec9a9348e21f57e
  experiment_tag: 0_alpha=7.3865e+05,u_hidden_dim=13,u_hidden_hidden_dim=9,u_layers=11,u_rate=4.0725e-05,v_hidden_dim=17,v_layers=3,v_

Trial name,status,loc,alpha,u_hidden_dim,u_hidden_hidden_dim,u_layers,u_rate,v_hidden_dim,v_layers,v_rate,iter,total time (s),Loss
train_f9627_00000,RUNNING,172.28.0.2:314,738645,13,9,11,4.07248e-05,17,3,3.01163e-05,5,31.8837,1.97666e+10
train_f9627_00001,RUNNING,172.28.0.2:315,552.21,8,12,11,1.35432e-06,5,6,4.70487e-05,4,24.5563,8.56528e+06
train_f9627_00002,PENDING,,9635.6,11,13,9,0.00154148,9,2,4.69799e-05,,,


Result for train_f9627_00001:
  Loss: 11324790.0
  date: 2021-01-27_11-16-08
  done: false
  experiment_id: 7fc901853453456588329fbe1b363afd
  experiment_tag: 1_alpha=552.21,u_hidden_dim=8,u_hidden_hidden_dim=12,u_layers=11,u_rate=1.3543e-06,v_hidden_dim=5,v_layers=6,v_rate=4.7049e-05
  hostname: fc555e4772b4
  iterations_since_restore: 5
  node_ip: 172.28.0.2
  pid: 315
  should_checkpoint: true
  time_since_restore: 32.54852080345154
  time_this_iter_s: 7.992223501205444
  time_total_s: 32.54852080345154
  timestamp: 1611746168
  timesteps_since_restore: 0
  training_iteration: 5
  trial_id: f9627_00001
  
(pid=315) 40 tensor([2758742.5000]) tensor([-8.4849])
Result for train_f9627_00000:
  Loss: 24379611136.0
  date: 2021-01-27_11-16-15
  done: false
  experiment_id: a2ee3c2d0cce484bbec9a9348e21f57e
  experiment_tag: 0_alpha=7.3865e+05,u_hidden_dim=13,u_hidden_hidden_dim=9,u_layers=11,u_rate=4.0725e-05,v_hidden_dim=17,v_layers=3,v_rate=3.0116e-05
  hostname: fc555e4772b4
  iteration

Trial name,status,loc,alpha,u_hidden_dim,u_hidden_hidden_dim,u_layers,u_rate,v_hidden_dim,v_layers,v_rate,iter,total time (s),Loss
train_f9627_00000,RUNNING,172.28.0.2:314,738645,13,9,11,4.07248e-05,17,3,3.01163e-05,6,39.5473,2.43796e+10
train_f9627_00001,RUNNING,172.28.0.2:315,552.21,8,12,11,1.35432e-06,5,6,4.70487e-05,5,32.5485,1.13248e+07
train_f9627_00002,PENDING,,9635.6,11,13,9,0.00154148,9,2,4.69799e-05,,,


(pid=314) 50 tensor([4.5775e+09]) tensor([-8.6515])
